# Multiple Views

Restart the server:

In [5]:
fuser -k 7000/tcp
python manage.py runserver 7000 &

7000/tcp:            35940
[1] 36226


Now add a new model in `tennis/models.py`:

In [6]:
cat << EOF > tennis/models.py
from django.db import models

class Match(models.Model):
    homeTeam = models.CharField(max_length=20)
    homeScore = models.IntegerField(default=0)
    awayTeam = models.CharField(max_length=20)
    awayScore = models.IntegerField(default=0)

    def play(self, home, score1, away, score2):
        self.homeTeam = home
        self.homeScore = score1
        self.awayTeam = away
        self.awayScore = score2
EOF

Traceback (most recent call last):
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/django/core/management/base.py", line 413, in run_from_argv
    self.execute(*args, **cmd_options)
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/django/core/management/commands/runserver.py", line 75, in execute
    super().execute(*args, **options)
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/django/core/management/base.py", line 459, in execute
    output = self.handle(*args, **options)
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/django/core/management/commands/runserver.py", line 82, in handle
    if not settings.DEBUG and not settings.ALLOWED_HOSTS:
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/django/conf/__init__.py", line 81, in __getattr__
    self._setup(name)
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/django/conf/__init__.py", line

Now we need to create tables in our database corresponding to our new models.  To do that, run the following command:

In [4]:
python manage.py migrate

Traceback (most recent call last):
  File "/home/chris/workspace/python-course/src/38 Django/manage.py", line 22, in <module>
    main()
  File "/home/chris/workspace/python-course/src/38 Django/manage.py", line 18, in main
    execute_from_command_line(sys.argv)
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/django/core/management/__init__.py", line 442, in execute_from_command_line
    utility.execute()
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/django/core/management/__init__.py", line 436, in execute
    self.fetch_command(subcommand).run_from_argv(self.argv)
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/django/core/management/base.py", line 405, in run_from_argv
    parser = self.create_parser(argv[0], argv[1])
  File "/home/chris/.pyenv/versions/3.10.15/lib/python3.10/site-packages/django/core/management/base.py", line 368, in create_parser
    self.add_arguments(parser)
  File "/home/chris/.pyenv/

: 1

In [ ]:
p manage.py makemigrations
p manage.py migrate

Migrations are Django’s way of propagating changes you make to your models (adding a field, deleting a model, etc.) into your database schema.

Note that by default we are using the `sqlite3` database:

In [ ]:
cat mysite/settings.py | grep ENGINE

The `migrate` command has now created a new class in `tennis\apps.py` 

In [ ]:
cat tennis/apps.py

This new class has to be published in the global `mysite/settings.py` file.  
This is a large file, but the relevent section is as follows:

In [ ]:
sed -n '/^INSTALLED_APPS/,/^]/p' mysite/settings.py

Now add our `tennis.apps.TennisConfig` class to this list:

In [ ]:
sed -i -e "/INSTALLED_APPS/ a \    'tennis.apps.TennisConfig'," mysite/settings.py

The relevant part of this file now reads:

In [ ]:
sed -n '/^INSTALLED_APPS/,/^]/p' mysite/settings.py

Now tell Django that you’ve made some changes to your models (in this case, you’ve made new ones) and that you’d like the changes to be stored as a migration:

In [ ]:
python manage.py makemigrations tennis

Let's take a moment to see the migrations Django has created:

In [ ]:
python manage.py sqlmigrate tennis 0001

Now, run migrate again to create those model tables in your database:

In [ ]:
python manage.py migrate &

Before we use these new classes, let's check our webapp is still working:

In [ ]:
firefox http:localhost:7000/tennis &

Now return to the project root

In [ ]:
cd "$projectRoot"
pwd